In [1]:
# Replace part of the OUTPUT_PATH to create a new folder 
# for the detection results

OUTPUT_PATH = "Temp/LoanApplications_Offline/"
NEW_OUTPUT_PATH = "Temp/LoanApplications_Offline__DETECTION/"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# My packages
from source import parse_mxml as pm
from source import log_representation as lr
from source import plots as plts
from source import drift_detection as dd
from source import drift_localization as dl
from source import offline_streaming_clustering as off_sc

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import hdbscan
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed, parallel_backend

import random
random.seed(42)

import os
import glob

import gc
gc.enable()

from scipy.spatial import distance
from sklearn.base import clone as sk_clone 

from copy import deepcopy

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

In [3]:
def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

def if_any(string, lista):
    # If the string contains any of the values
    # from the list 'lista'
    for l in lista:
        if l in string:
            return True
    return False

In [4]:
# List log files
logs = insensitive_glob(r"../../../../../../../Datasets/Business_Process_Drift_Logs/Logs/*/*k.MXML")
logs = [x.replace('\\', '/') for x in logs if "2.5" not in x]
# logs = [x for x in logs if "2.5" not in x]

In [5]:
# reference objects and map them to strings in dict 
# used in further methods
objects = {
    "model": {
#         "kmeans__k=6": KMeans(n_clusters=6, random_state=42),
#         "kmeans__k=3": KMeans(n_clusters=3, random_state=42),
#         "kmeans__k=2": KMeans(n_clusters=2, random_state=42),
        "DBSCAN__eps=05ms=5": DBSCAN(eps=0.5, min_samples=5, metric='euclidean'),
#         "DBSCAN__eps=1ms=4": DBSCAN(eps=1, min_samples=4, metric='euclidean'),
#         "DBSCAN__eps=2ms=3": DBSCAN(eps=2, min_samples=3, metric='euclidean'),
        "HDBSCAN__noparams": hdbscan.HDBSCAN(gen_min_span_tree=True, allow_single_cluster=True)
    },
    
    "representation": {
        "activity_binary": lambda x: lr.get_binary_representation(lr.get_traces_as_tokens(x)),
        "activity_frequency": lambda x: lr.get_frequency_representation(lr.get_traces_as_tokens(x)),
        "transitions_binary": lambda x: lr.get_binary_transitions_representation(lr.get_traces_as_tokens(x)),
        "transitions_frequency": lambda x: lr.get_frequency_transitions_representation(lr.get_traces_as_tokens(x)),
        "activity_transitions_frequency": lambda x: pd.concat([lr.get_frequency_transitions_representation(lr.get_traces_as_tokens(x)), lr.get_frequency_representation(lr.get_traces_as_tokens(x))],axis=1),
        "activity_transitions_binary": lambda x: pd.concat([lr.get_binary_transitions_representation(lr.get_traces_as_tokens(x)), lr.get_binary_representation(lr.get_traces_as_tokens(x))],axis=1)
    }
}

In [6]:
# change patterns and they supported representations
activity_binary_drifts = ["cb", "cf", "cm", "fr", "pm", "re", "rp"]
activity_frequency_drifts = activity_binary_drifts + ["cp", "lp"]
transitions_binary_drifts = activity_frequency_drifts + ["cd", "pl", "sw"]
transitions_frequency_drifts = transitions_binary_drifts
activity_transitions_frequency_drifts = transitions_binary_drifts
activity_transitions_binary_drifts = transitions_binary_drifts

### Pipeline Offline Clustering

In [7]:
def read_file_and_run_clustering_pipeline(args, return_result=False):
    """
    Read an event log file, represent it into a feature vector space and
    run the trace clustering method over windows. This method outputs results
    as gzip csv files into the "OUTPUT_PATH" folder, or return the result 
    as DataFrame when return_result = True.
    
    Parameters:
    -----------
        args (dict): Dictionary with the parameters and the log_file path
            requiring the following keys:
                example = {
                    'log': <PATH TO LOG_FILE>,
                    'representation': <KEY TO REPRESENTATIONS IN 'objects'>,
                    'parameters': [{
                        'model': <KEY TO MODEL IN 'objects'>, 
                        'sliding_window': <WHETHER TO USE SLIDING WINDOW>,
                        'window_size': <SIZE OF TRACE WINDOW TO USE>,
                        'sliding_step': <STEP OF SLIDING WINDOW>
                    }
        return_result (bool): Whether to return the result as DataFrame
            
    """
    # Create final dataset
    all_results = pd.DataFrame()
    
    # Treat file name to structure size and log type
    split = args["log"].split("/")
    
    # Parse change pattern name
    tipo_mudanca = split[-2]
    log_name = split[-1][:-5]

    # Parse size of the event_log
    log_size = int(float(log_name.replace(tipo_mudanca, "").replace("k", "")) * 1000)


    try:
        # Read log and apply trace representation technique
        log_read = pm.all_prep(open(args["log"]))
        df = objects["representation"][args["representation"]](log_read)
        
        # Create metric dataset
        all_metrics = pd.DataFrame() 
        
        for p in args["parameters"]:
            print(p)
            
            # If file does not exists, run trace clustering step and export file
            all_metrics = off_sc.run_offline_clustering_window(
                sk_clone(objects["model"][p["model"]]),
                p["window_size"],
                df,
                p["sliding_window"],
                sliding_step=p['sliding_step']
            )
            
            # Set up true drifts indexes and append
            y_true = list(range(int(len(df)/10), len(df), int(len(df)/10)))
            all_metrics["y_true"] = all_metrics.apply(lambda x: y_true, axis = 1)
            
            all_metrics = all_metrics.reset_index()
            
            # Identify columns
            all_metrics['tipo_mudanca'] = tipo_mudanca
            all_metrics['log_size'] = str(log_size)
            all_metrics['model'] = p["model"]
            all_metrics['representation'] = args["representation"]
            all_metrics['window_size'] = str(p["window_size"])
            all_metrics['sliding_window'] = str(p["sliding_window"])
            
        
            # Append results in final dataset
            all_results = all_results.append(all_metrics)
            
            gc.collect()
            
    except Exception as e:
        raise e
    
    all_results = all_results.reset_index(drop=True)
    return all_results

#### Run pipeline for specific case(s)

In [8]:
# read_file_and_run_clustering_pipeline({
#     'log': logs[0],
#     'representation': 'activity_binary',
#     'parameters': [{
#         'model': 'DBSCAN__eps=05ms=5', 
#         'sliding_window': False,
#         'window_size': 150,
#         'sliding_step': 1
#     }]
# }, return_result=True)

In [9]:
#read_file_and_run_clustering_pipeline({
#     'log': logs[0],
#     'representation': 'activity_binary',
#     'parameters': [{
#         'model': 'kmeans__k=6', 
#         'sliding_window': False,
#         'window_size': 200,
#         'sliding_step': 1
#     }]
# }, return_result=True)


### Run Experiments with several parameters combinations

In [10]:
# Trace clustering parameters
grid_parameters = list(ParameterGrid({
    "sliding_window": [False]    
    ,"window_size": [75, 100, 125, 150, 175, 200]
    ,"sliding_step": [1]
    ,"model": [
#         'kmeans__k=6',
#         'kmeans__k=3',
#         'kmeans__k=2',
        "DBSCAN__eps=05ms=5",
#         "DBSCAN__eps=1ms=4",
#         "DBSCAN__eps=2ms=3",
        "HDBSCAN__noparams"
    ] 
}))

# Trace vector representations
grid_logs = list(ParameterGrid([
    {
        "log": [x for x in logs if if_any(x, activity_binary_drifts)],
        "representation": ["activity_binary"]
    },
    {
        "log": [x for x in logs if if_any(x, activity_frequency_drifts)],
        "representation": ["activity_frequency"]
    },
    {
        "log": [x for x in logs if if_any(x, transitions_binary_drifts)],
        "representation": ["transitions_binary"]
    },
    {
        "log": [x for x in logs if if_any(x, transitions_frequency_drifts)],
        "representation": ["transitions_frequency"]
    },
    {
        "log": [x for x in logs if if_any(x, activity_transitions_frequency_drifts)],
        "representation": ["activity_transitions_frequency"]
    },
    {
        "log": [x for x in logs if if_any(x, activity_transitions_binary_drifts)],
        "representation": ["activity_transitions_binary"]
    }
]))

# Combining all parameters
combs = []
for x in grid_logs:
    dic = x.copy()
    dic['parameters'] = grid_parameters 
    
    combs.append(dic)

len(combs), len(grid_parameters), len(combs) * len(grid_parameters) 

(192, 12, 2304)

#### Run parallely

In [12]:
final_resp = pd.DataFrame()
final_resp = final_resp.append(Parallel(n_jobs=-2)(
    delayed(read_file_and_run_clustering_pipeline)(comb,return_result=False) for comb in tqdm_notebook(combs)
))

final_resp = final_resp.reset_index(drop=True)

try:
    os.makedirs(OUTPUT_PATH)
except:
    pass
final_resp.to_pickle(OUTPUT_PATH + "clustering_results" + '.pickle.gzip', compression="gzip")

final_resp.shape

(140800, 55)

### Detection Pipeline

In [13]:
# Drift detection parameters
drift_config = list(ParameterGrid([
    {
        "rolling_window": [3, 4]
        ,"std_tolerance": [1, 1.5, 2]
        ,"min_tol": [0.003, 0.005, 0.007, 0.01] 
    }
]))
len(drift_config)

24

In [14]:
def drift_detect_pipeline(drift_comb, return_results=False):
    # Runs the drift detection for every feature
    results = []
    for col in drift_comb.select_dtypes(include=np.number).columns:
        if (col not in ["i","test_id"]):
#         if (col in ["Silhouette"]):
#         if (col not in ["k"] and not col.startswith("diff") ) or col in ["diff_centroids"]:

            drift_comb_temp = drift_comb.copy()
            drift_comb_temp["measure"] = col
            
            print(col)
            
            # Define index as iteration number (i)
            drift_comb_temp.set_index("i", inplace=True)
    
            detected_drifts, info = dd.detect_concept_drift(
                drift_comb_temp, 
                col,
                int(drift_comb_temp["rolling_window"].iloc[0]),
                float(drift_comb_temp["std_tolerance"].iloc[0]),
                float(drift_comb_temp["min_tol"].iloc[0])
            )    
            
            # Calculate classification metrics
            metrics_results = dd.get_metrics(
                detected_drifts,
                drift_comb_temp["y_true"].iloc[0],
                int(drift_comb_temp["window_size"].iloc[0])
            )
            
            drift_comb_temp.reset_index(inplace=True)
            info = pd.DataFrame(info, index=drift_comb_temp.index)
            metrics_results = pd.DataFrame([metrics_results], index=drift_comb_temp.index)

            results = pd.concat([drift_comb_temp,metrics_results,info], axis=1)

        gc.collect()
        
    return results

#### Run pipeline for specific case(s)

In [18]:
# drift_detect_pipeline(drifts_combinations[drifts_combinations["test_id"]==0])

k
Silhouette
DBi
calinski_harabasz_score
validity_index
avg_dist_between_centroids
std_dist_between_centroids
avg_radius
std_radius
avg_dist_intra_cluster
std_dist_intra_cluster
avg_skewness
std_skewness
avg_cluster_std
std_cluster_std
diff_k
adjusted_rand_score
adjusted_mutual_info_score
homogeneity_score
completeness_score
v_measure_score
fowlkes_mallows_score
diff_Silhouette
diff_DBi
diff_calinski_harabasz_score
diff_validity_index
diff_centroids
std_diff_centroids
total_MSE
avg_MSE
count_non_zero_MSE
diff_avg_dist_between_centroids
diff_std_dist_between_centroids
diff_radius
diff_dist_intra_cluster
diff_skewness
diff_cluster_std
relative_validity
diff_relative_validity


,i,k,y_pred,Silhouette,DBi,calinski_harabasz_score,validity_index,centroids,avg_dist_between_centroids,std_dist_between_centroids,volume_list,radius_list,dist_intra_cluster_list,skewness_list,cluster_std_list,avg_radius,std_radius,avg_dist_intra_cluster,std_dist_intra_cluster,avg_skewness,std_skewness,avg_cluster_std,std_cluster_std,diff_k,adjusted_rand_score,adjusted_mutual_info_score,homogeneity_score,completeness_score,v_measure_score,fowlkes_mallows_score,diff_Silhouette,diff_DBi,diff_calinski_harabasz_score,diff_validity_index,diff_centroids,std_diff_centroids,total_MSE,avg_MSE,count_non_zero_MSE,diff_avg_dist_between_centroids,diff_std_dist_between_centroids,diff_volume,diff_radius,diff_dist_intra_cluster,diff_skewness,diff_cluster_std,y_true,tipo_mudanca,log_size,model,representation,window_size,sliding_window,relative_validity,diff_relative_validity,min_tol,rolling_window,std_tolerance,test_id,measure,Precision,Recall,F1,Delay,Correct_Predictions,Support,Drifts_Found,Resp,lowers,uppers,means
0,100,4,"[0, 0, 1, 0, 0, 4, 2, 0, 2, 0, 2, 0, 1, 3, 0, 3, 2, 0, 0, 0, 1, 0, 1, 4, 0, 0, 0, 4, 3, 0, 0, 1,...",0.792254,0.823908,124.598200,0.787734,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]...",2.181134,0.521810,"[40, 16, 13, 13, 18]","[0.0, 0.0, 0.0, 0.0, 2.313406979295224]","[0.0, 0.0, 0.0, 0.0, 1.843581617841975]","[0.31980107453341566, -0.5455447255899809, -0.5455447255899807, -0.5455447255899806, -0.62493242...","[0.4937279747182558, 0.48237638894271995, 0.48237638894272, 0.48237638894272, 0.4772445792538752]",0.462681,0.925363,0.368716,0.737433,-0.388353,0.355410,0.483620,0.005431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",cb,10000,DBSCAN__eps=05ms=5,activity_binary,100,False,NaN,NaN,0.003,3,1,0,diff_relative_validity,0,0.0,0.0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,[],"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",NaN,NaN,NaN
1,200,5,"[0, 2, 0, 3, 2, 3, 1, 3, 3, 4, 3, 5, 3, 0, 0, 1, 0, 4, 5, 3, 0, 0, 0, 4, 0, 5, 4, 0, 0, 4, 2, 0,...",0.877051,0.697294,193.409123,0.852468,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]...",2.272879,0.455361,"[43, 10, 11, 16, 14, 6]","[0.0, 1.9287301521985911, 0.0, 0.0, 0.0, 0.0]","[0.0, 1.894811374526966, 0.0, 0.0, 0.0, 0.0]","[0.3198010745334156, -0.5455447255899807, -0.5455447255899807, -0.5455447255899809, -0.545544725...","[0.49372797471825586, 0.48237638894272, 0.48237638894272, 0.48237638894271995, 0.48237638894272,...",0.321455,0.718795,0.315802,0.706155,-0.401320,0.322495,0.484268,0.004230,1.0,-0.028486,0.015998,0.092984,0.088587,0.090732,0.224782,0.084796,-0.126614,68.810923,0.064735,0.212568,0.425136,0.000000,0.000000,0.0,0.091746,-0.066449,0,0.000000,0.000000,0.000000,0.000000e+00,"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",cb,10000,DBSCAN__eps=05ms=5,activity_binary,100,False,NaN,NaN,0.003,3,1,0,diff_relative_validity,0,0.0,0.0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0]",0,[],"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",NaN,NaN,NaN
2,300,5,"[0, 0, 1, 0, 2, 0, 5, 1, 3, 2, 0, 4, 2, 3, 5, 3, 0, 0, 3, 0, 0, 4, 0, 2, 2, 5, 0, 4, 5, 0, 0, 4,...",0.873144,0.633062,235.809613,0.869987,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]...",2.405256,0.642471,"[45, 8, 11, 14, 9, 13]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.9791815892720934]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.5246453230912211]","[0.3198010745334158, -0.10540925533894575, -0.5455447255899808, -0.5455447255899806, -0.54554472...","[0.4937279747182558, 0.4993069989739546, 0.48237638894272, 0.48237638894272006, 0.48237638894272...",0.329864,0.737597,0.254108,0.568202,-0.379380,0.381236,0.483332,0.012372,0.0,-0.013144,-0.032185,0.064362,0.064851,0.064606,0.243365,-0.003906,-0.064232,42.400490,0.017519,0.397150,0.565915,2.272860,0.119624,13.0,0.132377,0.187110,"[2, -2, 0, -2,

#### Run parallely

In [16]:
# Read file
clusterings = pd.read_pickle(OUTPUT_PATH + "clustering_results.pickle.gzip", compression='gzip')
clusterings.shape

(140800, 55)

In [17]:
# Combining all parameters
drifts_combinations = pd.DataFrame()
for config in drift_config:
    temp = clusterings.copy()
    temp["min_tol"] = str(config["min_tol"])
    temp["rolling_window"] = str(config["rolling_window"])
    temp["std_tolerance"] = str(config["std_tolerance"])
    drifts_combinations = drifts_combinations.append(temp)

drifts_combinations = drifts_combinations.reset_index(drop=True)
drifts_combinations["test_id"] = drifts_combinations.groupby([
    'tipo_mudanca','log_size','model','representation','window_size','sliding_window'
    ,'min_tol','rolling_window','std_tolerance'
    ]).ngroup()
drifts_combinations

,i,k,y_pred,Silhouette,DBi,calinski_harabasz_score,validity_index,centroids,avg_dist_between_centroids,std_dist_between_centroids,volume_list,radius_list,dist_intra_cluster_list,skewness_list,cluster_std_list,avg_radius,std_radius,avg_dist_intra_cluster,std_dist_intra_cluster,avg_skewness,std_skewness,avg_cluster_std,std_cluster_std,diff_k,adjusted_rand_score,adjusted_mutual_info_score,homogeneity_score,completeness_score,v_measure_score,fowlkes_mallows_score,diff_Silhouette,diff_DBi,diff_calinski_harabasz_score,diff_validity_index,diff_centroids,std_diff_centroids,total_MSE,avg_MSE,count_non_zero_MSE,diff_avg_dist_between_centroids,diff_std_dist_between_centroids,diff_volume,diff_radius,diff_dist_intra_cluster,diff_skewness,diff_cluster_std,y_true,tipo_mudanca,log_size,model,representation,window_size,sliding_window,relative_validity,diff_relative_validity,min_tol,rolling_window,std_tolerance,test_id
0,75,4,"[0, 0, 1, 0, 0, 4, 2, 0, 2, 0, 2, 0, 1, 3, 0, 3, 2, 0, 0, 0, 1, 0, 1, 4, 0, 0, 0, 4, 3, 0, 0, 1,...",0.778878,0.840834,82.536541,0.777725,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]...",2.179280,0.510825,"[30, 13, 7, 11, 14]","[0.0, 0.0, 0.0, 0.0, 2.201576429631777]","[0.0, 0.0, 0.0, 0.0, 1.9078924663386718]","[0.3198010745334157, -0.5455447255899807, -0.5455447255899808, -0.5455447255899807, -0.579257469...","[0.4937279747182558, 0.48237638894272, 0.48237638894272, 0.48237638894272, 0.4802622382758289]",0.440315,0.880631,0.381578,0.763157,-0.379218,0.349753,0.484224,0.004822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",cb,10000,DBSCAN__eps=05ms=5,activity_binary,75,False,NaN,NaN,0.003,3,1,120
1,150,4,"[0, 4, 0, 1, 2, 3, 0, 0, 3, 0, 4, 3, 3, 4, 0, 4, 0, 4, 0, 0, 3, 1, 0, 3, 2, 0, 2, 0, 1, 2, 1, 4,...",0.862840,0.629739,203.783333,0.811190,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]...",2.234556,0.553111,"[31, 11, 9, 13, 11]","[0.0, 0.0, 0.0, 0.0, 1.8272501129310708]","[0.0, 0.0, 0.0, 0.0, 1.427373534245738]","[0.31980107453341583, -0.5455447255899809, -0.5455447255899808, -0.5455447255899807, -0.76891740...","[0.4937279747182557, 0.48237638894272, 0.48237638894272, 0.48237638894272, 0.4666972990084803]",0.365450,0.730900,0.285475,0.570949,-0.417150,0.378495,0.481511,0.008613,0.0,-0.005500,-0.003063,0.081617,0.081618,0.081618,0.242518,0.083962,-0.211095,121.246792,0.033465,0.382283,0.480912,4.021572,0.211662,13.0,0.055276,0.042285,"[1, -2, 2, 2, -3]",0.028024,0.046180,0.007194,0.000037,"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",cb,10000,DBSCAN__eps=05ms=5,activity_binary,75,False,NaN,NaN,0.003,3,1,120
2,225,5,"[0, 2, 0, 0, 3, 0, 5, 4, 4, 3, 4, 0, 2, 0, 2, 0, 1, 0, 0, 1, 5, 0, 0, 3, 3, 0, 5, 0, 0, 2, 0, 0,...",0.895834,0.666191,193.636549,0.862175,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]...",2.368069,0.666584,"[32, 7, 12, 11, 5, 8]","[0.0, 0.0, 0.0, 0.0, 0.0, 2.038688303787511]","[0.0, 0.0, 0.0, 0.0, 0.0, 1.5090459872429685]","[0.31980107453341583, -0.1054092553389457, -0.5455447255899808, -0.5455447255899809, -0.54554472...","[0.49372797471825575, 0.49930699897395464, 0.48237638894272, 0.48237638894272, 0.482376388942719...",0.339781,0.759774,0.251508,0.562388,-0.358398,0.357245,0.484999,0.009413,1.0,0.008077,-0.029349,0.079933,0.075259,0.077526,0.252026,0.032994,0.036452,-10.146784,0.050985,0.318424,0.390090,0.000000,0.000000,0.0,0.133512,0.113474,0,0.000000,0.000000,0.000000,0.000000,"[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000]",cb,10000,DBSCAN__eps=05ms=5,activity_binary,75,False,NaN,NaN,0.003,3,1,120
3,300,4,"[4, 0, 2, 4, 0, 0, 2, 0, 3, 0, 4, 4, 3, 3, 1, 0, 0, 0, 4, 4, 0, 4, 0, 0, 0, 3, 2, 0, 0, 4, 3, 4,...",0.747398,0.850170,83.510329,0.721680,"[[1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0,

In [ ]:
drifts_combinations_grouped = drifts_combinations.groupby("test_id")
final_drift_detection = pd.DataFrame()
final_drift_detection = final_drift_detection.append(Parallel(n_jobs=-2)(
    delayed(drift_detect_pipeline)(drift_comb) for i,drift_comb in tqdm_notebook(drifts_combinations_grouped)
))

final_drift_detection = final_drift_detection.reset_index(drop=True)

try:
    os.makedirs(NEW_OUTPUT_PATH)
except:
    pass
final_drift_detection.to_pickle(NEW_OUTPUT_PATH + "drift_detections_results" + '.pickle.gzip', compression="gzip")


final_drift_detection.shape

### Consolidate results

# Results

In [ ]:
# Read file
detections = pd.read_pickle(NEW_OUTPUT_PATH + "drift_detections_results.pickle.gzip", compression='gzip')
detections.shape

In [ ]:
model = list(objects["model"].keys())[1]
print(model)

# Filter tests
detections_filtered = detections[detections["model"]==model]
print(detections_filtered.shape)

all_results = detections_filtered.set_index("test_id").groupby("test_id").first()

# Group results by all params and get the mean F1 and Delay among all scenarios
# In other words, what combination of params got the best results in all scenarios

params = ["min_tol", "rolling_window", "std_tolerance", "window_size", "measure", "representation"]
scenarios = ["tipo_mudanca", "log_size"]
validation_metrics = ["F1","Delay"]#,"Support","Precision","Recall"]

all_results_grouped = all_results.groupby(params)[validation_metrics].agg(['mean','std','count'])
all_results_grouped.columns = all_results_grouped.columns.map('_'.join)
# all_results_grouped = all_results_grouped[all_results_grouped['F1_count']>=len(activity_binary_drifts)]
all_results_grouped.sort_values(["F1_mean","Delay_mean"], ascending=[False,True], inplace=True)
all_results_grouped = all_results_grouped.reset_index()
all_results_grouped.head(50)

In [ ]:
best_results_by_representation = all_results_grouped.reset_index().groupby("representation").first().reset_index()
best_results_by_representation

In [ ]:
best_result = pd.merge(left=all_results.reset_index()
            , right=best_results_by_representation
            , how='inner', on=params).set_index("test_id")
results_table = best_result.pivot_table(values=["F1"]#,"Delay"]
                                        , index=["tipo_mudanca"]
                                        , columns=["representation"]
                                        , aggfunc='mean')\
                                    .sort_index(axis='columns',level=[1,0], ascending=[True,False])
results_table

In [ ]:
results_table['F1'].plot(kind='bar', figsize=(20,10), fontsize=20)
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
results_table_logsize = best_result.pivot_table(values=["F1"]#,"Delay"]
                                                , index=["tipo_mudanca"]
                                                , columns=["representation", "log_size"]
                                                , aggfunc='mean')\
                                    .sort_index(axis='columns',level=[1,0], ascending=[True,False])
results_table_logsize

In [ ]:
for row in range(0,len(best_result)):
    best_result_drifts = detections_filtered[detections_filtered["test_id"]==best_result.index[row]]
    best_result_drifts.set_index("i", inplace=True)
    
    plts.plot_deteccao_drift(
        best_result_drifts,
        best_result_drifts.iloc[0]['measure'],
        best_result_drifts.iloc[0]['Drifts_Found'],
        best_result_drifts.iloc[0]['Resp'],
        best_result_drifts['means'],
        best_result_drifts['lowers'],
        best_result_drifts['uppers'],
        save_png=""
    )
    plt.title("Log: " + best_result_drifts.iloc[0]["tipo_mudanca"] + str(best_result_drifts.iloc[0]["log_size"]) 
                  + " - Rep: " + best_result_drifts.iloc[0]["representation"] 
                  + " - Metric: " + best_result_drifts.iloc[0]["measure"]
                  + " - F1: " + str(round(best_result_drifts.iloc[0]["F1"],2))
              , size=30)
    plt.show()

In [ ]:
for row in range(0,len(best_result)):

    best_result_log = [x for x in logs if best_result.iloc[row]["tipo_mudanca"] in x 
         and str(best_result.iloc[row]["log_size"]/1000) in 
                       str(float(x.split("/")[-1][:-5].replace("k", "").replace(x.split("/")[-2],"")))]
    
    run_df = off_sc.run_offline_clustering_window(
        objects["model"][best_result.iloc[row]['model']],
        int(best_result.iloc[row]['window_size']),
        objects["representation"][best_result.iloc[row]['representation']](pm.all_prep(open(best_result_log[0]))),
        sliding_window=False,
        sliding_step=1
    )

    drifts, info = dd.detect_concept_drift(
        run_df,
        best_result.iloc[row]['measure'],
        rolling_window=best_result.iloc[row]['rolling_window'],
        std_tolerance=best_result.iloc[row]['std_tolerance'],
        min_tol=best_result.iloc[row]['min_tol']
    )

    plts.plot_deteccao_drift(
        run_df,
        best_result.iloc[row]['measure'],
        best_result.iloc[row]['Drifts_Found'],
        best_result.iloc[row]['Resp'],
        info['means'],
        info['lowers'],
        info['uppers'],
        save_png=""
    )
    plt.title("Log: " + best_result.iloc[row]["tipo_mudanca"] + str(best_result.iloc[row]["log_size"]) 
                  + " - Rep: " + best_result.iloc[row]["representation"] 
                  + " - Metric: " + best_result.iloc[row]["measure"]
                  + " - F1: " + str(round(best_result.iloc[row]["F1"],2))
              , size=30)
    plt.show()

In [ ]:
# all_results.to_excel('Resultados_PL.xlsx')